In [2]:
from hw1 import *
import plotly.express as px
import pandas as pd
from IPython import display

In [33]:
agent_params = dict(
        action_n = 6,
        state_n = 500,
        smoothing_type = None,
        l_value = 0.1
    )

fit_params = dict(
        n_iterations = 30,
        n_trajectories = 100,
        n_packs = 20,
        n_steps = 200,
        stochastic_env = False,
        q = 0.6,
    )

ДЗ 1. Алгоритм Cross-Entropy без сглаживания

In [70]:
no_smoothing_rewards = []
smoothing = 'no_smoothing'
for n_trajectories in [100, 300, 500, 1000]:
    for l_val in [1, 2, 3, 4, 5]:
        for q_val in [0.5, 0.7, 0.9]:
            print(f'n_trajectories={n_trajectories} l_val={l_val} q_val={q_val}')
            agent_params['l_value'] = l_val
            agent_params['smoothing_type'] = None
            
            fit_params['n_trajectories'] = n_trajectories
            fit_params['q'] = q_val
            
            env = gym.make('Taxi-v3')
            agent = CrossEntropyAgent(**agent_params)

            fit_rewards = fit(env, agent, **fit_params)
            reward_records = [{'smoothing': smoothing, 'l_value': l_val, 'q_value': q_val, 'n_trajectories': n_trajectories,
                            'iteration': i, 'reward': r} for i, r in enumerate(fit_rewards)]
            no_smoothing_rewards.extend(reward_records)
            
            display.clear_output(wait=True)

n_trajectories=1000 l_val=5 q_val=0.9
iteration: 0/30
iteration: 10/30
iteration: 20/30


In [71]:
df = pd.DataFrame.from_records(no_smoothing_rewards)

In [81]:
df_grpd = df.groupby(['n_trajectories', 'iteration']).agg({'reward': 'mean'}).reset_index()
fig = px.line(df_grpd, x='iteration', y='reward', color='n_trajectories')
fig.update_layout(
    title = 'Влияние количества траекторий на среднюю награду',
    xaxis_title = 'Итерация',
    yaxis_title = 'Средняя награда'
)

По графикам награды видно, что количество траекторий, участвующих в обучении, оказывает ключевую роль в увеличении средней награды. При этом уже на 10-й итерации величина награды практически достигает своего пика.

In [87]:
df_grpd = df.groupby(['n_trajectories', 'iteration', 'q_value']).agg({'reward': 'mean'}).reset_index()
fig = px.line(df_grpd[df_grpd['n_trajectories'] == 1000], x='iteration', y='reward', color='q_value')
fig.update_layout(
    title = 'Скорость и качество обучения, в зависимости от параметра q (100 траекторий)',
    xaxis_title = 'Итерация',
    yaxis_title = 'Средняя награда'
)

Высокий параметр q негативно влияет на величину награды, сильно уменьшая количество траекторий для обучения. C q == 0.5 агент обучается медленее, но достигает более высокой награды.
Исходя из того, что и увеличение количества траекторий, и уменьшение q приводит к росту награды, можно сделать выводы, что количество элитных траекторий, применяемых для обучения, играет решающую роль, позволяя при обучении использовать большее разнообразие.

ДЗ 2. Алгоритм Кросс-Энтропии с типами сглаживания "policy" и "laplace"

Обучение с применением сглаживания требует больше итераций. С n_trajectories=500, l_val=0.1 и q_val=0.7 достаточно 100 итераций.
Пробую подбирать другие параметры.

In [102]:
policy_rewards = []
smoothing = 'policy'
for n_iter in [100]:
    for n_trajectories in [100, 500, 1000]:
        for l_val in [0.01, 0.1, 0.5]:
            for q_val in [0.5, 0.7]:
                print(f'n_trajectories={n_trajectories} l_val={l_val} q_val={q_val}')
                agent_params['l_value'] = l_val
                agent_params['smoothing_type'] = smoothing
                fit_params['q'] = q_val
                fit_params['n_trajectories'] = n_trajectories
                fit_params['n_iterations'] = n_iter
                
                env = gym.make('Taxi-v3')
                agent = CrossEntropyAgent(**agent_params)

                fit_rewards = fit(env, agent, **fit_params)
                reward_records = [{'smoothing': smoothing, 'l_value': l_val, 'q_value': q_val, 'n_trajectories': n_trajectories,
                                'iteration': i, 'reward': r} for i, r in enumerate(fit_rewards)]
                policy_rewards.extend(reward_records)
                
                display.clear_output(wait=True)

n_trajectories=1000 l_val=0.5 q_val=0.7
iteration: 0/100
iteration: 10/100
iteration: 20/100
iteration: 30/100
iteration: 40/100
iteration: 50/100
iteration: 60/100
iteration: 70/100
iteration: 80/100
iteration: 90/100


In [103]:
df2 = pd.DataFrame.from_records(policy_rewards)

In [108]:
df_grpd = df2.groupby(['n_trajectories', 'iteration', 'l_value']).agg({'reward': 'mean'}).reset_index()
fig = px.line(df_grpd, x='iteration', y='reward', line_group='l_value', color='n_trajectories')
fig.update_layout(
    title = 'Влияние количества траекторий и l_value на среднюю награду',
    xaxis_title = 'Итерация',
    yaxis_title = 'Средняя награда',
)

Из графика обучения можно сделать вывод, что при обучении с policy_smoothing количество траекторий играет уже не такую не решающую роль как без сглаживания. l_value сильно влияет на скорость обучения. Видно, что при l=0.01 модель за 100 итераций практически не обучилась.
В качестве оптимального можно брать 500 траекторий и l = 0.25.
Проанализируем влияние q:

In [114]:
df_grpd = df2.groupby(['n_trajectories', 'iteration', 'l_value', 'q_value']).agg({'reward': 'mean'}).reset_index()
fig = px.line(df_grpd[(df_grpd['l_value'] == 0.5) & (df_grpd['n_trajectories'] == 500)], 
              x='iteration', y='reward', color='q_value')
fig.update_layout(
    title = 'Влияние q-value на среднюю награду',
    xaxis_title = 'Итерация',
    yaxis_title = 'Средняя награда',
)

меньший q_value, как и в обучении без сглаживания, показывает лучшие результаты 

In [115]:
laplace_rewards = []
smoothing = 'laplace'
for n_trajectories in [300, 500, 1000]:
    for l_val in [0.1, 0.5, 1, 2]:
        for q_val in [0.5]:
            print(f'n_trajectories={n_trajectories} l_val={l_val} q_val={q_val}')
            agent_params['l_value'] = l_val
            agent_params['smoothing_type'] = smoothing
            fit_params['q'] = q_val
            fit_params['n_trajectories'] = n_trajectories
            
            env = gym.make('Taxi-v3')
            agent = CrossEntropyAgent(**agent_params)

            fit_rewards = fit(env, agent, **fit_params)
            reward_records = [{'smoothing': smoothing, 'l_value': l_val, 'q_value': q_val, 'n_trajectories': n_trajectories,
                            'iteration': i, 'reward': r} for i, r in enumerate(fit_rewards)]
            laplace_rewards.extend(reward_records)

            display.clear_output(wait=True)

n_trajectories=1000 l_val=2 q_val=0.5
iteration: 0/100
iteration: 10/100
iteration: 20/100
iteration: 30/100
iteration: 40/100
iteration: 50/100
iteration: 60/100
iteration: 70/100
iteration: 80/100
iteration: 90/100


In [116]:
df3 = pd.DataFrame.from_records(laplace_rewards)
df_grpd = df3.groupby(['n_trajectories', 'iteration', 'l_value']).agg({'reward': 'mean'}).reset_index()
fig = px.line(df_grpd, x='iteration', y='reward', line_group='l_value', color='n_trajectories')
fig.update_layout(
    title = 'Влияние количества траекторий на среднюю награду',
    xaxis_title = 'Итерация',
    yaxis_title = 'Средняя награда',
)

В сглаживании Лапласа результат снова зависит от количества траекторий и величины l. Лучшие результаты достигаются при l=0.1 и количестве траекторий 1000.


Сравним все три алгоритма

In [129]:
final_rewards = []
for smoothing in [None, 'policy', 'laplace']:
    for n_trajectories in [1000]:
        for l_val in [0.25, 0.5]:
            for q_val in [0.5]:
                print(f'n_trajectories={n_trajectories} l_val={l_val} q_val={q_val}')
                agent_params['l_value'] = l_val
                agent_params['smoothing_type'] = smoothing
                fit_params['q'] = q_val
                fit_params['n_trajectories'] = n_trajectories
                
                env = gym.make('Taxi-v3')
                agent = CrossEntropyAgent(**agent_params)

                fit_rewards = fit(env, agent, **fit_params)
                reward_records = [{'smoothing': smoothing, 'l_value': l_val, 'q_value': q_val, 'n_trajectories': n_trajectories,
                                'iteration': i, 'reward': r} for i, r in enumerate(fit_rewards)]
                final_rewards.extend(reward_records)

                display.clear_output(wait=True)

n_trajectories=1000 l_val=0.5 q_val=0.5
iteration: 0/100
iteration: 10/100
iteration: 20/100
iteration: 30/100
iteration: 40/100
iteration: 50/100
iteration: 60/100
iteration: 70/100
iteration: 80/100
iteration: 90/100


In [130]:
df_final = pd.DataFrame.from_dict(final_rewards)
df_final['smoothing'].fillna('no_smoothing', inplace=True)
df_grpd = df_final.groupby(['smoothing', 'iteration', 'l_value']).agg({'reward': 'mean'}).reset_index()
fig = px.line(df_grpd, x='iteration', y='reward', line_group='l_value', color='smoothing')
fig.update_layout(
    title = 'Влияние количества траекторий на среднюю награду',
    xaxis_title = 'Итерация',
    yaxis_title = 'Средняя награда',
)

Вывод: в данной задаче все три метода обучения показали себя вполне хорошо. Метод без сглаживания обучается быстрее всех, но тем самым подвержден риску неоптимального выбора действия.
Сглаживание позволяет этого избежать.
Сглаживание по политике обучается медленнее всех, но показывает лучший результат

ДЗ 3. Стохастичесчкая среда

In [138]:
policy_rewards = []
smoothing = 'policy'
fit_params['stochastic_env'] = True

for n_iter in [50]:
    for n_trajectories in [2000]:
        for l_val in [0.5]:
            for n_packs in [100]:
                for q_val in [0.5]:
                    print(f'n_trajectories={n_trajectories} l_val={l_val} q_val={q_val}, n_packs={n_packs}')
                    agent_params['l_value'] = l_val
                    agent_params['smoothing_type'] = smoothing
                    fit_params['q'] = q_val
                    fit_params['n_trajectories'] = n_trajectories
                    fit_params['n_iterations'] = n_iter
                    fit_params['n_packs'] = n_packs
                    
                    env = gym.make('Taxi-v3')
                    agent = CrossEntropyAgent(**agent_params)

                    fit_rewards = fit(env, agent, **fit_params)
                    reward_records = [{'smoothing': smoothing, 'l_value': l_val, 'q_value': q_val, 'n_trajectories': n_trajectories,
                                    'iteration': i, 'reward': r} for i, r in enumerate(fit_rewards)]
                    policy_rewards.extend(reward_records)
                    
                    display.clear_output(wait=True)

n_trajectories=2000 l_val=0.5 q_val=0.5, n_packs=100
stochastic env
iteration: 0/50
	total delivered: 9501/200000
	mean penalties: 63.903955
	mean steps: 196.51734
	mean reward: -770.65533


Че-т не учится нифига